In [200]:
import random

In [209]:
def generate_string_01(length,max_consecutive_zeros):
    string='1'
    consecutive_zeros=0
    for _ in range(length-1):
        bit=random.choice('01')
        if bit=='0':
            consecutive_zeros+=1
            if consecutive_zeros>max_consecutive_zeros:
                bit='1'
                consecutive_zeros=0
        else:
            consecutive_zeros=0
        
        
        string+=bit
    return string

In [210]:
#ocena je fitnes samo
def calculate_fitness(rating): 
    return rating

In [211]:
def create_first_gen(population_size,string_length,max_consecutive_zeros):
    return [generate_string_01(string_length,max_consecutive_zeros) for _ in range(population_size)]

In [212]:
def crossover(parent1,parent2):
    random_pos=random.randint(1,len(parent1)-1)

    child1=parent1[:random_pos]+parent2[random_pos:]
    child2=parent2[:random_pos]+parent2[random_pos:]

    return child1,child2

    

In [213]:
def mutate(individual,mutation_rate,max_consecutive_zeros):
    mutated=''
    consecutive_zeros=0
    for bit in individual:
        if random.random() < mutation_rate:
            #ako je TRENUTNI bit 0 i ima vise 0 za redom nego max,menjamo ga u 1
            if bit=="0" and consecutive_zeros>max_consecutive_zeros:
                bit="1"
                mutated+=str(bit)
                #moramo da resetujemo brojac uzastopnih sad jer nam je sad 1,pa da krene ispocetka od sledeceg
                consecutive_zeros=0  
            else: 
     
                mutated += "1" if bit == "0" else "0"
                consecutive_zeros = consecutive_zeros + 1 if bit == "1" else 0
       
        else:
            mutated+=str(bit)
            if bit=="1":
                consecutive_zeros=0
            else: 
                consecutive_zeros+=1


    mutated= '1'+ mutated[1:]
        
    return mutated
                

In [214]:

#biramo random roditelje iz populacije,roulette
def select_parents(population,fitness_scores):
    total_fitness=sum(fitness_scores)
    selection_probabilities=[score/total_fitness for score in fitness_scores]
    #ovdje bira k=2 iz population koje ce da vrati kao roditelje,na osnovu skora koji je dobio
    selected_indices=random.choices(range(len(population)),weights=selection_probabilities,k=2)
    parent1=population[selected_indices[0]]
    parent2=population[selected_indices[1]]

    while True:
        if parent1==parent2:
            continue

        else:
            return parent1,parent2
    
   

In [215]:
def genetic_algorithm(string_length,population_size,mutation_rate,max_consecutive_zeros):   
    #ovde ga samo inicijalizuje
    population=create_first_gen(population_size,string_length,max_consecutive_zeros)
    best_strings_rated_5 = [] 
   
    overall_best_string = None
    overall_best_fitness = 0

    
    best_string= None
    best_fitness=0
    second_best_string=None
    second_best_fitness=0

    #dok ga ne prekine user da ide
    while True:
        population_fitness={individual : 0 for individual in population}

        for individual in population_fitness:
            population_fitness[individual]=calculate_fitness(int(input(f"Rate {individual}")))

             #ovde da nadjem overall najbolji,>= jer zelim da cuvam najnovijeg
            if population_fitness[individual] >= overall_best_fitness:
                overall_best_string = individual
                overall_best_fitness = population_fitness[individual]

            if population_fitness[individual]==5:
                best_strings_rated_5.append(individual)
            
        sorted_population=sorted(population_fitness.items(),key=lambda item: item[1],reverse=True)
        best_string, best_fitness = sorted_population[0]
        second_best_string, second_best_fitness = sorted_population[1]

        user_input = input(f"Best string: {best_string}, Best fitness: {best_fitness}\n"
                           f"Second best string: {second_best_string}, Second best fitness: {second_best_fitness}\n"
                           f"Overall best string: {overall_best_string},Overall best fitness: {overall_best_fitness}\n"
                           "Continue to the next generation? (y/n): ")
        if user_input.lower()!='y':
                break

        #dalje biramo roditelje za sledecu gen
        fitness_scores= [population_fitness[individual] for individual in population]
 
        parent1 , parent2=select_parents(population,fitness_scores)
        child1,child2=crossover(parent1,parent2)
        child1 = mutate(child1,mutation_rate,max_consecutive_zeros)
        child2 = mutate(child2,mutation_rate,max_consecutive_zeros)

        
 
   
        population = [child1, child2] + population[2:]

       
    return overall_best_string,overall_best_fitness,best_strings_rated_5      



 
        
        

In [216]:
def save_strings_to_file(strings, filename):
    with open(filename, 'w') as file:
        for string in strings:
            file.write(string + '\n')


In [217]:
if __name__ == "__main__":
    string_length = int(input("unesi broj karaktera u stringu"))
    population_size = int(input("unesi population size"))
    mutation_rate = 0.7
    max_consecutive_zeros = int(input("unesi max broj uzastopnih nula"))
    best_string, best_fitness,rated_5 = genetic_algorithm(string_length, population_size, mutation_rate, max_consecutive_zeros)
    print(f"Best string: {best_string}, Best fitness: {best_fitness}")
    print("Strings rated with a 5:")
    for string in rated_5:
        print(string)

    filename = "best_strings_rated_5.txt"
    save_strings_to_file(rated_5, filename)
    print(f"saved to '{filename}'")



unesi broj karaktera u stringu 10
unesi population size 5
unesi max broj uzastopnih nula 3
Rate 1011011011 2
Rate 1000110001 3
Rate 1000101011 4
Rate 1100011101 5
Rate 1001110100 5
Best string: 1100011101, Best fitness: 5
Second best string: 1001110100, Second best fitness: 5
Overall best string: 1001110100,Overall best fitness: 5
Continue to the next generation? (y/n):  y
Rate 1100111110 3
Rate 1110001110 2
Rate 1000101011 4
Rate 1100011101 5
Rate 1001110100 3
Best string: 1100011101, Best fitness: 5
Second best string: 1000101011, Second best fitness: 4
Overall best string: 1100011101,Overall best fitness: 5
Continue to the next generation? (y/n):  n


Best string: 1100011101, Best fitness: 5
Strings rated with a 5:
1100011101
1001110100
1100011101
saved to 'best_strings_rated_5.txt'
